In [ ]:
#default_exp seq2seq.namegen

# Import


In [ ]:
#export
from typing import List
from fastai.text.all import *
from dotmap import DotMap
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Dataset

Creating dataset from names list

In [ ]:
#export 
def get_dataset(names_list: List[str])-> Datasets:
    src_tfms = [lambda x: ['xxbos'] + list(x), Numericalize()]
    len_tfms = [lambda x: torch.tensor(len(x)+1, dtype=torch.int32)]
    #tgt_tfms = [lambda x: list(x)[1:]]

    dsrc = Datasets(names_list, tfms=[src_tfms, len_tfms], splits=RandomSplitter(valid_pct=0.1)(names_list))
    return dsrc

In [ ]:
file_lists=['/notebooks/nlp_deeplearning/charmodel/data/first_names.all.txt']

names_list = []
with open(file_lists[0],'r') as file:
    for name in file.read().splitlines()[1:]:
        filtered_name = re.sub(r'\W+', '', name)
        names_list.append(filtered_name.upper())

In [ ]:
#export 
@ItemTransform
def after_item(obj):
    return (obj[0][:], obj[0][1:], obj[1])

def pad_input_chunk_new(samples, n_inp=2,**kwargs):
    "Pad `samples` by adding padding by chunks of size `seq_len`"
    
    max_len = max([len(s[n]) for s in samples for n in range(n_inp)])
    padeds = [[pad_chunk(s[n],pad_len=max_len,**kwargs) for n in range(n_inp) ] for s in samples]
    
    return [(*p, *s[n_inp:]) for p, s in zip(padeds, samples)]

# Training

Here we'll define function to train the model. This will be an API to deal with user input too.

In [ ]:
#export
def train_from_scratch(names_list: List[str], hparams:DotMap):
    dsrc = get_dataset(names_list)
    dls = dsrc.dataloaders(after_item=after_item, before_batch=pad_input_chunk_new, bs=32, n_inp=2)

    trainer = pl.Trainer(fast_dev_run=False, auto_lr_find='learning_rate',gpus=1,
                    callbacks=[EarlyStopping(monitor='val_loss',patience=5)],
                    )
    return trainer

@patch
def gen_name(trainer:pl.Trainer, initial_char='A'):
    model = trainer.model
    model = model.to(device)
    return model.generate(initial_char)

In [ ]:
#export
def load_from_checkpoint(trainer:pl.Trainer, checkpoint:str)->pl.Trainer:
    # load from checkpoint
    trainer.model.load_from_checkpoint('example.ckpt')
    return trainer


In [ ]:

from nbdev.export import notebook2script
notebook2script()

Converted model.ipynb.
Converted namegen.ipynb.
Converted run.ipynb.
